In [44]:
import pandas as pd
from scipy.spatial import KDTree
from itertools import chain
from collections import Counter
from webcolors import (
    CSS3_HEX_TO_NAMES,
    hex_to_rgb,
)

color_names = pd.read_csv('./datasets/colornames.csv')
df = pd.read_csv("./datasets/output.csv")
df.head()

,Unnamed: 0,saturation,brightness,entropy,sharpness,contrast,frame_nr,colorfulness,col_palette,movie_id
0,1,48.620459,41.318256,4.449458,624.281683,62.055632,0,38.521991,"[(28, 60, 48), (198, 196, 168), (129, 99, 87),...",tt12316630.mp4
1,2,13.313068,44.688463,3.037550,28.117523,63.074007,49,11.809414,"[(56, 60, 52), (165, 181, 161), (190, 207, 186...",tt12316630.mp4
2,3,25.738858,43.961366,3.121975,28.100927,77.422133,109,13.806843,"[(38, 47, 47), (217, 230, 210), (160, 174, 155...",tt12316630.mp4
3,4,15.886666,51.800640,3.277400,18.451270,56.934361,133,15.459579,"[(104, 113, 99), (212, 226, 198), (175, 190, 1...",tt12316630.mp4
4,5,14.471347,58.068028,2.979327,6.835990,61.155942,160,17.518376,"[(95, 106, 93), (219, 232, 202), (184, 199, 17...",tt12316630.mp4


In [2]:
# Makes a list of unique values from a list
def unique(list1):
    # Init null list
    unique_list = []

    for x in list1:
        if x not in unique_list:
            unique_list.append(x)
            #print(x)
    return unique_list

In [46]:
color_names_dict = color_names.to_dict()

In [47]:
print(color_names)

                              name      hex
0                          100 Mph  #c93f38
1               18th Century Green  #a59344
2                   1975 Earth Red  #7b463b
3               1989 Miami Hotline  #dd3366
4      20000 Leagues Under the Sea  #191970
...                            ...      ...
29200                      Zumthor  #cdd5d5
29201                  Zunda Green  #6bc026
29202                         Zuni  #008996
29203                  Zürich Blue  #248bcc
29204                 Zürich White  #e6e1d9

[29205 rows x 2 columns]


In [3]:
col_df = df.copy()
col_df = col_df.replace('\[','', regex=True)
col_df = col_df.replace('\]','', regex=True)

col_df = col_df.replace('\),',').', regex=True)

print(col_df['col_palette'])

col_df[['dom_col1', 'dom_col2', 'dom_col3', 'dom_col4', 'dom_col5']] = col_df['col_palette'].str.split('.',0,expand=True)
col_df = col_df.drop(columns=['Unnamed: 0', 'saturation', 'brightness', 'entropy', 'sharpness', 'contrast', 'frame_nr', 'colorfulness'])
unique_movie_id_list = unique(col_df['movie_id'])
col_df.head()

0        (28, 60, 48). (198, 196, 168). (129, 99, 87). ...
1        (56, 60, 52). (165, 181, 161). (190, 207, 186)...
2        (38, 47, 47). (217, 230, 210). (160, 174, 155)...
3        (104, 113, 99). (212, 226, 198). (175, 190, 16...
4        (95, 106, 93). (219, 232, 202). (184, 199, 175...
                               ...                        
58335    (85, 34, 7). (227, 197, 83). (237, 224, 172). ...
58336    (4, 4, 4). (217, 217, 217). (83, 83, 83). (124...
58337    (4, 4, 4). (217, 217, 217). (84, 84, 84). (124...
58338    (4, 4, 4). (217, 217, 217). (83, 83, 83). (124...
58339    (4, 4, 4). (218, 218, 218). (84, 84, 84). (124...
Name: col_palette, Length: 58340, dtype: object


,col_palette,movie_id,dom_col1,dom_col2,dom_col3,dom_col4,dom_col5
0,"(28, 60, 48). (198, 196, 168). (129, 99, 87). ...",tt12316630.mp4,"(28, 60, 48)","(198, 196, 168)","(129, 99, 87)","(144, 172, 149)","(131, 156, 157)"
1,"(56, 60, 52). (165, 181, 161). (190, 207, 186)...",tt12316630.mp4,"(56, 60, 52)","(165, 181, 161)","(190, 207, 186)","(140, 153, 136)","(155, 164, 156)"
2,"(38, 47, 47). (217, 230, 210). (160, 174, 155)...",tt12316630.mp4,"(38, 47, 47)","(217, 230, 210)","(160, 174, 155)","(129, 141, 125)","(140, 155, 156)"
3,"(104, 113, 99). (212, 226, 198). (175, 190, 16...",tt12316630.mp4,"(104, 113, 99)","(212, 226, 198)","(175, 190, 165)","(36, 47, 45)","(179, 197, 180)"
4,"(95, 106, 93). (219, 232, 202). (184, 199, 175...",tt12316630.mp4,"(95, 106, 93)","(219, 232, 202)","(184, 199, 175)","(165, 178, 155)","(172, 188, 180)"


In [50]:
def convert_rgb_to_names(rgb_tuple):
    # a dictionary of all the hex and their respective names in css3
    css3_db = CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []    
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)    
    distance, index = kdt_db.query(rgb_tuple)
    return names[index]

print(convert_rgb_to_names((255,1,2)))

TypeError: expected string or bytes-like object

In [5]:
print(type((255,1,2)))

<class 'tuple'>


In [28]:
''' 
for index, row in col_df.iterrows():
    try: 
        col_df['dom_col1'][index] = convert_rgb_to_names(eval(row['dom_col1']))
        col_df['dom_col2'][index] = convert_rgb_to_names(eval(row['dom_col2']))
        col_df['dom_col3'][index] = convert_rgb_to_names(eval(row['dom_col3']))
        col_df['dom_col4'][index] = convert_rgb_to_names(eval(row['dom_col4']))
        col_df['dom_col5'][index] = convert_rgb_to_names(eval(row['dom_col5']))
    except:
        print("Failed at index: ", index)
'''

Failed at index:  847


In [7]:
col_df = pd.read_csv('color_df.csv')
col_df.head()

,Unnamed: 0,col_palette,movie_id,dom_col1,dom_col2,dom_col3,dom_col4,dom_col5
0,0,"(28, 60, 48). (198, 196, 168). (129, 99, 87). ...",tt12316630.mp4,darkslategray,silver,dimgray,darkseagreen,lightslategray
1,1,"(56, 60, 52). (165, 181, 161). (190, 207, 186)...",tt12316630.mp4,darkslategray,darkgray,silver,gray,darkgray
2,2,"(38, 47, 47). (217, 230, 210). (160, 174, 155)...",tt12316630.mp4,darkslategray,gainsboro,darkgray,gray,lightslategray
3,3,"(104, 113, 99). (212, 226, 198). (175, 190, 16...",tt12316630.mp4,dimgray,lightgray,darkgray,darkslategray,silver
4,4,"(95, 106, 93). (219, 232, 202). (184, 199, 175...",tt12316630.mp4,dimgray,gainsboro,silver,darkgray,darkgray


In [39]:
def pool_colors(df, id):
    col1 = df['dom_col1'][df['movie_id'] == id]
    col2 = df['dom_col2'][df['movie_id'] == id]
    col3 = df['dom_col3'][df['movie_id'] == id]
    col4 = df['dom_col4'][df['movie_id'] == id]
    col5 = df['dom_col5'][df['movie_id'] == id]

    return list(chain(col1, col2, col3, col4, col5))

In [43]:
for x in unique_movie_id_list:
    col_pool = pool_colors(col_df, x)
    c = Counter(col_pool)
    print("Element with highest frequency:\n",c.most_common(5))
    #print(col_pool)

Element with highest frequency:
 [('black', 83), ('gray', 82), ('darkslategray', 81), ('dimgray', 64), ('darkgray', 60)]
Element with highest frequency:
 [('black', 167), ('dimgray', 125), ('gray', 108), ('darkslategray', 103), ('darkolivegreen', 61)]
Element with highest frequency:
 [('black', 95), ('dimgray', 64), ('gray', 63), ('slategray', 44), ('darkolivegreen', 23)]
Element with highest frequency:
 [('black', 155), ('gray', 99), ('darkslategray', 84), ('dimgray', 82), ('darkgray', 54)]
Element with highest frequency:
 [('black', 135), ('darkolivegreen', 35), ('darkslategray', 32), ('teal', 30), ('darkcyan', 30)]
Element with highest frequency:
 [('black', 93), ('gray', 86), ('dimgray', 81), ('darkslategray', 81), ('darkolivegreen', 45)]
Element with highest frequency:
 [('dimgray', 85), ('black', 74), ('gray', 73), ('darkslategray', 60), ('darkolivegreen', 50)]
Element with highest frequency:
 [('black', 60), ('slategray', 37), ('darkslategray', 33), ('dimgray', 27), ('lightslate